In [1]:
import numpy as np
import pandas as pd

import arviz as az

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

/home/ec2-user/miniconda/lib/python3.7/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")


Loading BokehJS ...

## Problem 8.2: Microtubule catastrophe, 40 pts

_Note: This problem is best done after the lecture November 22._

In this problem, we use data from [Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, **147**, 1092-1103, 2011](https://doi.org/10.1016/j.cell.2011.10.037). The authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state.  In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules in a single-molecule [TIRF assay](https://en.wikipedia.org/wiki/Total_internal_reflection_fluorescence_microscope) by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin. We will carefully analyze the data and make some conclusions about the processes underlying microtubule catastrophe.

In the file `gardner_mt_catastrophe_only_tubulin.csv` (which you can download [here](../data/gardner_mt_catastrophe_only_tubulin.csv)), we have observed catastrophe times of microtubules with different concentrations of tubulin. To start with, we will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe. We will consider three models for microtubule catastrophe.

- Model 1: The time to catastrophe is Exponentially distributed.
- Model 2: The time to catastrophe is Gamma distributed.
- Model 3: The time to catastrophe is Weibull distributed.

Note that these descriptions are for the likelihood; we have not specified priors.


**a)  Describe the three models in words. Give physical descriptions of the meanings of their parameters. Describe how these models are related to each other. Tutorial 3c will be useful.** 

<br />



- Model 1: The time to catastrophe is Exponentially distributed.

This suggests that the occurance of catastrophe is a Poisson process, so it is a "rare event" that requires multiple subprocesses to lead it it. The parameter for the process, if it is exponential, $\beta$, represents the characteristic rate of catastrophe, that is how often catastrophe happens in a certain amount of time. It can also be parametrized as $\tau=1/\beta$, the characteristic catastrophe time, which fits what we are given in our data. The Exponential distribution is a special case of the Gamma distribution where $\alpha = 1$ and a special case of the Weibull distribution where $\alpha = 1$ and $\sigma=1/\beta$



- Model 2: The time to catastrophe is Gamma distributed.

This suggests that the occurance of catastrophe represents a specific number of occurances of a Poisson process, that is a discrete number of steps that occur at the same rate must occur for catastrophe to occur. There are two parameters for this distribution, $\alpha$ and $\beta$, where $\alpha$ is the number of arrivals (or "steps") required to trigger catastrophe, and $\beta$ is the rate of the arrivals. Thus, the characteristic catastrophe time is given by $\alpha/\beta$. 


- Model 3: The time to catastrophe is Weibull distributed.

This suggests that the likelihood of catastrophe is dependent on the amount of time it has been since the last catastrophe, so the longer it has been since the last catastrophe, the more likely it is that catastrophe will occur. There are two parameters for this distribution, $\alpha$ which defines how the probability changes over time, and $\sigma$ which is the characteristic catastrophe time.

**b) Perform parameter estimates for the respective models and make model comparisons. Comment on what this means with respect to our understanding of how microtubule catastrophe works.**

We first load in our data and come up with our priors for the parameters of the three distributions. We don't have much prior knowledge so we will keep them simple (normal distribution).

*Exponential(tao)*

tao ~ normal(700, 100)

*Gamma($\alpha$, $\beta$)*

alpha ~ normal(10, 3)

beta ~ normal(10, 3)

*Weibull($\alpha$, $\sigma$)*

$\alpha$ ~ normal(3, 0.05)

$\sigma$ ~ normal(10, 3)


In [5]:
df = pd.read_csv('/home/ec2-user/data/gardner_mt_catastrophe_only_tubulin.csv', comment = "#")

In [82]:
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35.0,25.0,50.0,60.0
1,40.000,45.0,40.0,60.0,75.0
2,40.000,50.0,40.0,60.0,75.0
3,45.429,50.0,45.0,75.0,85.0
4,50.000,55.0,50.0,75.0,115.0


In [63]:
prior_sm1 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m1.stan')
prior_sm2 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m2.stan')
prior_sm3 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m3.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_084aff62b7267c6e5172712f7ec900d9 NOW.


Using cached StanModel.
Using cached StanModel.


/home/ec2-user/miniconda/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmpaxme2ynu/stanfit4anon_model_084aff62b7267c6e5172712f7ec900d9_3780854271197521839.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [65]:
# Store input parameters in a dictionary so stan can access them
data = dict(N=692)

# Generate samples
samples_gen1 = prior_sm1.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen2 = prior_sm2.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen3 = prior_sm3.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)

# Store samples in a dataframe
df_gen1 = bebi103.stan.to_dataframe(samples_gen1, diagnostics=False)
df_gen2 = bebi103.stan.to_dataframe(samples_gen2, diagnostics=False)
df_gen3 = bebi103.stan.to_dataframe(samples_gen3, diagnostics=False)

# Let's look at one of the dataframes to make sure they look away
df_gen3.head()

,chain,chain_idx,warmup,uM_12[1],uM_12[2],uM_12[3],uM_12[4],uM_12[5],uM_12[6],uM_12[7],...,uM_12[686],uM_12[687],uM_12[688],uM_12[689],uM_12[690],uM_12[691],uM_12[692],alpha,sigma,lp__
0,1,1,0,376.467016,2433.847279,704.482308,688.777455,2815.672401,538.901780,382.496489,...,1350.075999,939.277877,653.673705,422.402776,171.514060,244.055300,116.110145,0.974421,601.314697,0.0
1,1,2,0,386.273425,52.656172,46.989560,15.284868,121.779870,261.973964,119.625049,...,358.216759,460.849834,372.569606,660.170422,34.257408,139.381623,103.196077,0.947059,275.936338,0.0
2,1,3,0,77.134755,6.365506,433.036646,91.854794,241.603278,290.041921,128.494516,...,139.364940,381.313546,395.150512,1238.805969,641.533338,163.931697,5.617199,0.876617,310.724479,0.0
3,1,4,0,675.688129,2.473518,1.622834,112.055645,5.006407,347.054713,22.162829,...,51.277000,536.158793,350.815329,343.692630,18.637853,273.963639,692.298251,0.856552,259.107412,0.0
4,1,5,0,784.965555,347.516735,1947.447523,805.131347,192.047915,1354.688035,140.459471,...,668.878276,1961.011696,656.232064,340.005927,236.669448,131.154538,388.288050,0.932283,501.457494,0.0


In [66]:
p = bebi103.viz.predictive_ecdf(samples_gen1, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 6000)
bokeh.io.show(p)

In [67]:
p = bebi103.viz.predictive_ecdf(samples_gen2, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 3000)
bokeh.io.show(p)

In [69]:
p = bebi103.viz.predictive_ecdf(samples_gen3, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 1000)
bokeh.io.show(p)

Our priors look reasonable, so now we can move on to creating our mcmc models and sampling.

In [104]:
sm1 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m1.stan')
sm2 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m2.stan')
sm3 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m3.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_beab730eb2c2e1d32001662e0dc5ff81 NOW.


Using cached StanModel.


/home/ec2-user/miniconda/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmp6c6_mwm9/stanfit4anon_model_beab730eb2c2e1d32001662e0dc5ff81_4496351417399247967.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


Using cached StanModel.


In [105]:
data = dict(N=len(df),
           uM_12=df['12 uM'].values.astype(float))

In [99]:
samples1 = sm1.sampling(data=data)
samples2 = sm2.sampling(data=data)
samples3 = sm3.sampling(data=data)

In [81]:
df_mcmc1 = bebi103.stan.to_dataframe(samples1, diagnostics=False, inc_warmup=False)
df_mcmc2 = bebi103.stan.to_dataframe(samples2, diagnostics=False, inc_warmup=False)
df_mcmc3 = bebi103.stan.to_dataframe(samples3, diagnostics=False, inc_warmup=False)

In [103]:
df_mcmc2.head()

,chain,chain_idx,warmup,alpha_,tao,beta_,log_like[1],log_like[2],log_like[3],log_like[4],...,uM_12_ppc[684],uM_12_ppc[685],uM_12_ppc[686],uM_12_ppc[687],uM_12_ppc[688],uM_12_ppc[689],uM_12_ppc[690],uM_12_ppc[691],uM_12_ppc[692],lp__
0,1,1,0,3.209207,120.434039,0.008303,-9.366675,-8.452889,-8.452889,-8.216800,...,442.920838,130.596781,544.982739,355.310202,611.936148,463.786329,273.267714,202.050533,291.451031,-4665.081082
1,1,2,0,3.282229,115.122574,0.008686,-9.417162,-8.474803,-8.474803,-8.231499,...,104.846017,672.417164,674.571295,487.266927,355.341179,387.371020,226.663029,390.530349,356.099498,-4664.955822
2,1,3,0,3.254664,115.634370,0.008648,-9.360329,-8.430348,-8.430348,-8.190344,...,633.464358,361.045293,201.137435,131.587049,43.832017,131.163949,237.005530,239.270318,271.472878,-4664.869484
3,1,4,0,3.032317,125.441544,0.007972,-9.032365,-8.196746,-8.196746,-7.981369,...,97.901472,340.156061,231.495826,915.474799,567.587235,280.079977,338.736988,318.629583,438.976129,-4664.912656
4,1,5,0,3.296510,111.339886,0.008982,-9.350940,-8.406294,-8.406294,-8.162775,...,298.654358,352.065525,484.729064,312.848749,1242.585268,286.294185,487.338594,367.568810,230.652195,-4666.260687


We now plot the post predictive checks to do model comparison.

In [75]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples1, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

We can see there is a systematic issue with using the exponential distribution becuase the two inflection points will always be in the wrong place.

In [76]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples2, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

The gamma distribution looks a lot more promising! Most of our values fall into that innermost confidence interval.

In [77]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples3, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

Ooof the weibull distribution model is also not great (better than exponential though). Only a few of our datapoints fall in that 99% confidence interval.

To confirm quantitatively, we compute the loo given the log likelihood and use the bebi103 compare function to calculate the loo and the weights.

In [78]:
bebi103.stan.compare({'exponential': samples1, 'gamma': samples2, 'weibull': samples3},
                     log_likelihood='log_like', ic='loo')

,loo,ploo,dloo,weight,se,dse,warning
gamma,9281.06,2.4435,0,0.958793,48.172,0,0
weibull,9321.51,2.01426,40.4515,1.04906e-13,43.0174,11.8882,0
exponential,9608.59,0.368836,327.524,0.0412069,31.9052,31.6414,0


We can see that gamma (Model 2) is the best distribution to fit our data because the weight is much higher (0.95 vs 10^-2 and 10^-13). Based on the visual analysis of the post predictive checks above,  this makes a lot of sense! 

**c) Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?**


We picked the gamma distribution model so we will now sample out of it for all 5 concentrations in the dataframe.

In [136]:
data12 = dict(N=len(df),
           uM_12=df['12 uM'].values.astype(float))
data7 = dict(N=len(df['7 uM'].dropna()),
           uM_12=df['7 uM'].dropna().values.astype(float))
data9 = dict(N=len(df['9 uM'].dropna()),
           uM_12=df['9 uM'].dropna().values.astype(float))
data10 = dict(N=len(df['10 uM'].dropna()),
           uM_12=df['10 uM'].dropna().values.astype(float))
data14 = dict(N=len(df['14 uM'].dropna()),
           uM_12=df['14 uM'].dropna().values.astype(float))

In [137]:
samples12 = sm2.sampling(data=data12)
samples7 = sm2.sampling(data=data7)
samples9 = sm2.sampling(data=data9)
samples10 = sm2.sampling(data=data10)
samples14 = sm2.sampling(data=data14)

In [144]:
df_mcmc12 = bebi103.stan.to_dataframe(samples12, diagnostics=False, inc_warmup=False)
df_mcmc7 = bebi103.stan.to_dataframe(samples7, diagnostics=False, inc_warmup=False)
df_mcmc9 = bebi103.stan.to_dataframe(samples9, diagnostics=False, inc_warmup=False)
df_mcmc10 = bebi103.stan.to_dataframe(samples10, diagnostics=False, inc_warmup=False)
df_mcmc14 = bebi103.stan.to_dataframe(samples14, diagnostics=False, inc_warmup=False)

In [145]:
df_mcmc7.head()

,chain,chain_idx,warmup,alpha_,tao,beta_,log_like[1],log_like[2],log_like[3],log_like[4],...,uM_12_ppc[600],uM_12_ppc[601],uM_12_ppc[602],uM_12_ppc[603],uM_12_ppc[604],uM_12_ppc[605],uM_12_ppc[606],uM_12_ppc[607],uM_12_ppc[608],lp__
0,1,1,0,2.506889,130.640517,0.007655,-7.414623,-7.112466,-6.991973,-6.991973,...,380.923333,571.297664,150.441407,150.251492,199.121873,118.013694,88.331744,435.943273,439.436333,-4016.858451
1,1,2,0,2.491148,128.262853,0.007796,-7.342040,-7.045258,-6.927132,-6.927132,...,345.211085,220.770430,433.263815,76.552839,426.147747,398.821303,312.952512,696.213904,771.629689,-4016.865376
2,1,3,0,2.565386,127.157777,0.007864,-7.471848,-7.157087,-7.031478,-7.031478,...,347.743413,337.021089,301.156604,643.100524,144.131658,284.125251,146.577844,148.654828,809.644611,-4017.080599
3,1,4,0,2.413682,134.151281,0.007454,-7.285159,-7.004423,-6.892748,-6.892748,...,528.842585,720.617331,194.671454,358.604575,76.972066,225.094274,154.533029,391.718831,205.016962,-4016.721874
4,1,5,0,2.497292,128.848920,0.007761,-7.364466,-7.065785,-6.946835,-6.946835,...,512.957409,597.650338,233.337818,93.146118,191.917812,232.859910,708.986196,253.646530,139.045781,-4016.771910


In [146]:
df_mcmc12.head()

,chain,chain_idx,warmup,alpha_,tao,beta_,log_like[1],log_like[2],log_like[3],log_like[4],...,uM_12_ppc[684],uM_12_ppc[685],uM_12_ppc[686],uM_12_ppc[687],uM_12_ppc[688],uM_12_ppc[689],uM_12_ppc[690],uM_12_ppc[691],uM_12_ppc[692],lp__
0,1,1,0,2.822633,133.374770,0.007498,-8.668053,-7.923874,-7.923874,-7.732610,...,224.423019,631.487008,252.475918,154.540439,600.262710,487.954473,502.498368,530.333422,509.987839,-4640.404736
1,1,2,0,2.766528,139.445658,0.007171,-8.642072,-7.919366,-7.919366,-7.733471,...,199.848601,304.199134,98.040945,528.595528,153.132966,211.119038,601.486515,333.727469,91.556001,-4640.800448
2,1,3,0,2.678962,139.393001,0.007174,-8.419968,-7.738459,-7.738459,-7.563723,...,304.267038,91.907525,98.984394,466.464838,246.063239,58.940676,185.737008,619.557668,244.280321,-4641.848416
3,1,4,0,2.825569,138.615605,0.007214,-8.777282,-8.027470,-8.027470,-7.834294,...,193.221039,305.622445,306.815059,435.321863,965.268617,627.210691,520.661052,208.866678,474.690143,-4641.048751
4,1,5,0,3.025368,123.201067,0.008117,-8.963766,-8.133588,-8.133588,-7.919883,...,226.788751,219.419506,720.947610,401.085898,756.387347,360.992851,348.496605,613.603442,559.555173,-4640.743096


In [138]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples12, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False,
                                          title = "12 uM"))

In [139]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples7, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['7 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "7 uM"))

In [141]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples9, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['9 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "9 uM"))

In [142]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples10, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['10 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "10 uM"))

In [143]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples14, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['14 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "14 uM"))

Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?